# 基于机器学习数据库飞速上线AI应用—智能实时预测Turbofan剩余寿命应用开发

剩余使用寿命（remaining useful life，RUL），指一个系统正常工作一段时间后,能够正常运转的时间。借助RUL,工程师可以安排维护时间、优化运行效率并避免计划外停机。因此,预测RUL是预测性维护计划中的首要任务。 
本次的任务就是开发一个通过机器学习模型进行剩余使用寿命预测的实时智能应用。我们使用NASA提供的[Turbofan Engine Degradation Simulation Data Set](https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/#turbofan)，作为训练集与测试集，开发一个智能预测硬件剩余寿命的应用

![turbo](https://ftp.bmp.ovh/imgs/2021/06/ea39f4d2b326d619.jpeg)


## 初始化环境
整个初始化过程包含安装[OpenMLDB](https://github.com/4paradigm/openmldb)，以及相关运行环境，初始化脚步可以参考init.sh

In [ ]:
!cd rul && sh init.sh

## 导入历史数据到[OpenMLDB](https://github.com/4paradigm/openmldb)

使用[OpenMLDB](https://github.com/4paradigm/openmldb)进行时序特征计算是需要历史数据的，所以我们将历史数据导入到fedb，以便实时推理可以使用历史数据进行特征推理，导入代码可以参考https://github.com/4paradigm/DemoApps/blob/main/predict-taxi-trip-duration-nb/demo/import.py 。

这里使用`test_FD004.txt`作为历史数据，有多个engine多次cycle的检测数据。

In [ ]:
!cd rul && python3 import.py

## 训练模型
模型训练需要训练数据，以下是生成使用的的代码

* 训练特征矩阵生成脚本代码 train.py
* 训练数据使用`train_FD004.txt`，多engine不同cycle的检测数据

最终生成模型

In [ ]:
!cd rul && python3 train.py ./fe.sql /tmp/model.txt

## 使用训练的模型搭建实时推理http服务

基于上一步生成的训练用特征矩阵，使用`RandomForestRegressor`进行训练。
得到的训练模型，再结合[OpenMLDB](https://github.com/4paradigm/openmldb)中的历史数据，搭建一个实时推理服务，整个推理服务代码参考predict_server.py

In [ ]:
!cd rul && sh start_predict_server.sh ./fe.sql 9887 /tmp/model.txt

## 通过http请求发送一个推理请求

In [ ]:
!cd rul && python3 predict.py